In [ ]:
# Por si alguien corre en python2
from __future__ import division

# Preparamos todo para correr
import numpy as np
from math import *
from matplotlib import pyplot as plt
%matplotlib inline
from scipy.stats import norm, binom, gamma, poisson, multivariate_normal
from sklearn import linear_model


## Ejercicio 1: Fitteando una recta ruidosa de manera Bayesiana.

Como primer ejemplo de regresión lineal, tratemos de fittear los parametros $a_{0}$, $a_{1}$ de una recta $f(x,\vec{a}) = a_0 + a_{1}x$ donde $x$ está en el intervalo $[-1,1)$.

* **A)** Genere un set artificial de datos usando, usando como valores $a_{1} = 0.5$, $a_{0}= -0.3$. Añada a las mediciones ruido gaussiano  $\epsilon \sim \mathcal{N}(\mu=0, \sigma = 0.2)$. 
Al terminar debería tener un conjunto de $N= 100$  pares $(x_n,t_n)$, con 
$t_n = f(x_n,\vec{a}) + \epsilon$.

* **B)** Como modelo usaremos una regresión lineal de la forma $y(x) = \omega_0 + \omega_1 x$. ¿Cuál base de funciones $\phi_j$ estamos usando? (recuerde que por convención $\phi_0(x)=1$). 
Escriba para estos la matriz de diseño.
$$\Phi = \begin{pmatrix}
\phi_0(x_1) & \phi_1(x_1)\\
\phi_0(x_2) & \phi_1(x_2)\\
\vdots & \vdots)\\
\phi_0(x_N) & \phi_1(x_N)\\
\end{pmatrix}$$

* **C)** Estamos interesados en encontrar los valores de $\omega_0$ y $\omega_1$ de nuestro modelo más probables, dado los datos que tenemos. Estos están dados por el máximo de nuestra distribución posterior. Si usamos priors uniformes en $\omega_0$ y $\omega_1$ para caracterizar nuestro desconocimiento, el máximo del posterior coincide con el de la verosimilitud y es lo que llamamos el *estimador de máxima verosimilitud*. De acuerdo a lo visto en la teórica, dicho valor esta dado por 
$$\begin{pmatrix}
\omega_0^\text{ML}\\
\omega_1^\text{ML}\\
\end{pmatrix} 
= \left(\Phi^T \Phi\right)^{-1}\Phi^T \,\vec{t}
$$
donde recordemos que $\vec{t} = \begin{pmatrix}t_1\\ \vdots \\ t_N\end{pmatrix}$ es el vector de los valores *target* medidos.

Nota: Para un modelo lineal con error gaussiano, este estimador de maxima verosimilitud coincide con lo que se conoce como la solución de *cuadrados mínimos*.

* **D)** Si en vez de utilizar un prior uniforme, utilizamos un prior gaussiano de la forma $p(\vec{\omega}) = \mathcal{N} (\vec{0},\alpha^{-1}{\bf 1})$ (es decir que $p(\vec{\omega}) = p(\omega_0) \times p(\omega_1)$ con cada $p(\omega_i) = \mathcal{N} (\mu=0,\sigma = \alpha)$ ). Calcule la verosimilitud y el posterior (prior*posterior normalizado) al usar solo 1 punto, 2 puntos, 3 puntos y todo el conjunto de 100 puntos.


Si se le complica hacerlo en forma numerica, puede utilizar la formula analitica dle posterior: Como este es un prior conjugado a la gaussiana en la teórica (o en su defecto, en el Bishop ecuacion 3.53), vimos que nuestro posterior es una gaussiana con valor medio
$$
\vec{m}_N = \beta {\bf S}_N \Phi^T \vec{t}\\
{\bf S}_N^{-1} = \alpha {\bf 1} + \beta \Phi^T \Phi
$$
donde $\beta$ es el parámetro de precisión del ruido gaussiano, que se supone conocido. En nuestro caso sería $\beta = (1/\sigma)^2 = (1/0.2)^2 = 25$. Para seleccionar la cantidad de puntos a considerar, puede usar slicing en $\vec{t}$ ( ``t[:N_puntos]``) y en $\Phi$ (``Phi[:N_puntos,:]``).

# Ejercicio 1 bis: Scikit-learn

Scikit-learn es una librería de machine learning con gran soporte para una multitud de algoritmos de análisis de datos, que nos permite acceder a ellos de una forma muy similar, obviando las diferencias técnicas entre sus implementaciones. En la práctica, nadie escribe sus propias funciones para algoritmos (a menos que sea un algoritmo novedoso que estás diseñando), sino que utilizamos las implementaciones provistas por este u otro paquete.

* **A)** Scikit-learn provee una clase LinearRegressor la cual permite obtener las soluciones de cuadrados mínimos en problemas lineales. Importe el paquete, y examine las distintas funciones leyendo la documentación provista en el paquete. Si no es suficiente, [visite la documentación on-line](https://scikit-learn.org/), para familiarizarse con como se utiliza.

In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
#descomente cada linea para ver la documentación.
#LinearRegression?
#reg.fit?
#reg.predict?
#reg.score?

* **B)** Utilizando los datos del ejercicio anterior, repita el inciso **C)** utilizando scikit-learn.

# Ejercicio 2: Algoritmo de Cuadrados Mínimos con Polinomios

El algoritmo de cuadrados mínimos consiste en hallar los parámetros que minimicen la distancia cuadrática entre los datos y mi ajuste $$ E_{D}(\vec{w})=\frac{1}{2}\left(t-\vec{w}^{T}\vec{\phi}(\vec{x})\right)^{2}.$$ Este puede verse como el estimador de máxima verosimilitud cuando modelamos los errores como gaussianos en un problema lineal.

 Considere el siguiente set de datos ``X``y ``T`` como dado:

In [ ]:
np.random.seed(42)
X=2*np.pi*np.random.rand(70)
T=np.asarray(list(map(lambda x: 1.0*np.sin(x)+norm.rvs(loc=0.0,scale=0.1),X)))
T.shape
plt.scatter(X,T)

* **A)** Considere la base de funciones los polinomios $\phi_j(x) = x^j$, con $j=1,\dots,M$. Calcule la matriz de diseño.

* **B)** Calcule la solución de cuadrados mínimos, utilizando la expresión dada en el inciso **C)** del **Ejercicio 1**, (la ecuación 3.15 del Bishop; ecuaciones normales). Grafíquela encima de los datos. Estudie como cambia para distintos tamaños de la base, i.e. distintos valores de $M$.

* **C)** Separe el set de datos en training y validation en una proporción de 0.8/0.2. En base a lo visto en la teoría, ¿observa overfitting? ¿a que se debe? ¿cómo lo reduciría? 

# Ejercicio 2 bis: Scikit-learn

Utilizar una base de funciones $\phi_j(x)$ es lo que en la jerga de data science se conoce como *feature extraction*. Transformamos nuestros datos $x$ en features $\tilde x = \phi_j(x)$, los cuales usamos para alimentar nuestros modelos (por ejemplo, el Regresor Lineal de los ejercicios anteriores. Este tipo de transformaciones es parte de lo que se conoce como *pre-procesado* de los datos.

* **A)** Importe de ``sklearn.preprocessing`` la clase ``PolynomialFeatures``. Examine su documentación para ver que es lo que hace.

* **B)** Utilizando ``PolynomialFeatures`` y ``LinearRegressor`` repita el inciso **B** del ejercicio anterior. 

Si le interesa aprender una forma mas elegante de tratar con preprocesado (que podría ser util si se realizan mas transformaciones previas), examine el importe la clase ``Pipeline`` de ``sklearn.pipeline``, e intente usarla con ayuda de [los ejemplos de la documentación](https://scikit-learn.org/stable/modules/linear_model.html#polynomial-regression-extending-linear-models-with-basis-functions).

# Ejercicio 3: SGD

Considere el siguiente dataset:

In [ ]:
X = 2 * np.random.rand(100, 1)
t = 4 + 3 * X + np.random.randn(100, 1)

plt.plot(X, t, "b.")
plt.xlabel("$x$", fontsize=18)
plt.ylabel("$t$", rotation=0, fontsize=18)
plt.show()

Queremos aproximar estos datos por un modelo lineal simple,

$y_{i} = w_{0} + w_{1}x_{i}$

**A)** Encuentre los valores de $w_{0}$ y $w_{1}$ que minimizan el error en cuadrados minimos

$E_{D}(\vec{w})=\sum_{n=1}^{N}E_{n}(\vec{w})=\frac{1}{2}\sum_{n=1}^{N}(t_{n}-y_{n}(\vec{w}))^{2}$

**B)** Una forma de aproximar la solución cerrada de forma iterativa consiste en aplicar el algoritmo de Descenso por Gradiente para minimizar la función $E_D(\vec{\omega})$:
$$
\vec{w}^{\tau+1}=\vec{w}^{\tau}-\eta\nabla E_{n}(\vec{w}^{\tau})\\
=\vec{w}^{\tau}+\eta(t_{n}-\vec{w}^{\tau T}\vec{\phi}(\vec{x}_n))\vec{\phi}(\vec{x}_{n}).
$$

Donde partimos de un valor $\vec{\omega}^0$ aleatorio que nos permite calcular de forma iterativa los $\vec{\omega}^\tau$ subsiguientes en función del parametro $\eta$ (también llamado *learning rate*).  Este algoritmo es muy util cuando los datos no entran en memoria o cuando la evaluacion es muy pesada, por lo que es mejor evaluar dato por dato. Este no es el caso pero podemos aprovechar para implementarlo y comparar con la solucion exacta.

Se recorre muchas veces el dataset y cada recorrida se denomina _epoca_ (epoch en ingles). En general, se recomienda elegir los datos al azar (pudiendo repetirse datos) en cada epoca. El algoritmo funciona de la siguiente manera:

* Inicializo $w_{0}$
* Epoca $j = 1,..,E$
 * Dato $i = 1,...,N$, elegido al azar de todo el dataset. Setteo $\eta_{i,j}$
  * Actualizo $w$ utilizando la Ecuacion con el dato $\vec{x}_{i}$ de la epoca $j$.

Inicialice $w_{0}$ al azar. Examine la evolucion del algoritmo en funcion de las epocas primero para un valor fijo de $\eta=0.2$ y luego siguiendo un _learning schedule_ en el que el valor de $\eta$ se actualiza de esta manera:

$\eta = \frac{t_{0}}{t+t_{1}}$ con $t = j\cdot N + i$ y $t_{0}, t_{1}$ hiperparametros. En principio tome $t_{0} = 10$ y $t_{1} = 50$. 

Compare con la solucion exacta. Que ventaja tiene la actualizacion del $\eta$?


 **C)** Se puede ver que incluir un prior normal centrado en cero para los parámetros, equivale a agregar a la función de cuadrados mínimos un término regularizador
$$E_{D}(\vec{w})=\frac{1}{2}\sum_{i=1}^{N}(t_{i}-\vec{w}^{T}\vec{\phi}(\vec{x_{i}}))^{2}+\frac{\kappa}{2}\vec{w}^{T}\vec{w}.$$
Esto se traduce, en el algoritmo recursivo, en la siguiente expresión
$$
\vec{w}^{\tau+1}=\vec{w}^{\tau}-\eta\left[(t_{n}-\vec{w}^{\tau T}\vec{\phi}(\vec{x}_n))\vec{\phi}(\vec{x}_{n}) + \kappa \vec{\omega}^{\tau}\right].
$$
Modifique el código del inciso anterior, y para un valor de $\kappa$ de su elección, estudie que sucede (puede comparar con el caso $\kappa=0$ que coincide con el inciso anterior).

# Ejercicio 3 bis: Scikit-Learn

* **A)** Examine la instancia de ``SGDRegressor``, y usando el argumento ``loss="squared_loss"`` utilicelo para resolver el inciso **B** anterior de forma iterativa (será util el argumento ``learning_rate=constant`` y ``eta0=`` para comparar con la realizada en el ejercicio anterior).

In [ ]:
from sklearn.linear_model import SGDRegressor
SGDRegressor?

# Ejercicio 4: Cross-validation para comparación de modelos

En este ejercicio utilizaremos el set de datos de precios inmobiliarios en el estado de Boston, que viene con scikit-learn.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline

import pandas as pd  

En la siguiente celda cargamos el dataset, y con el creamos un DataFrame de Pandas. Scikit-learn nos entrega un diccionario con los datos, el nombre de cada feature, el target, y una descripción general del dataset.

In [ ]:
from sklearn.datasets import load_boston

boston_dataset = load_boston()
boston = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
#añadimos el target como una columna llamada 'MEDV'
boston['MEDV'] = boston_dataset.target
boston.head()

Puede leer la descripción de cada feature ejecutando la siguiente celda

In [ ]:
print(boston_dataset.DESCR[296:1226])

El objetivo de este ejercicio es predecir el viviendas de Boston ("MEDV") a partir del resto de los features.

Para simplificar la tarea, solo consedere los dos features mas correlacionados con el target "MEDV". La siguiente lista, y los gráficos de correlación (ploteados por el muy util paquete ``seaborn``), pueden ayudarlo a elegir.

In [ ]:
from seaborn import pairplot
pairplot(boston, x_vars=boston.columns,y_vars='MEDV')
abs(boston.corr()['MEDV']).sort_values(ascending=False)

**A)**
Divida sus datos en Training /Test, usando en una proporcion de 0.8/0.2. 

In [ ]:
from sklearn.model_selection import train_test_split



**B)** Utilice validación cruzada y K-folding para comparar el desempeño de diferentes modelos lineales utilizando solamente los datos de Training. Utilice la métrica de desempeño ``scoring='neg_root_mean_squared_error'``. Los modelos a comparar son:
* Regresión lineal unidimensional utilizando cada uno de los features escogidos.
* Regresión lineal bidimensional utilizando ambos features.
* Regresión polinomial cuadrática unidimensional utilizando cada uno de los features escogidos.
* Regresión polinomial cuadrática  bidimensional utilizando ambos features.
En base a estos resultados, elija un modelo. 

In [ ]:
import sklearn.preprocessing as pp
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

**C)** Considere la regresión polinomial  cuadrática de los ultimos dos casos. Añada un término de regularización LASSO y RIDGE. Utilice validación cruzada para medir su desempeño, ¿cómo se compara a los casos anteriores? ¿Puede elegir los valores óptimos para la regularización?

In [ ]:
from sklearn.linear_model import Ridge, Lasso, RidgeCV, LassoCV


**D)**  (*) Considere una Regresión polinomial de grado ``M`` con regularización (LASSO o RIDGE). Utilice validación cruzada para obtener la mejor combinación de parámetros ``M`` y ``alpha`` (el regulador)

In [ ]:
from sklearn.model_selection import GridSearchCV


**E)**  (*) Opcional: Explore el dataset. Pruebe incluír mas features,  y explore las diversas funcionalidades vistas en clase: Estratificación, distintas funciones de base, etc.